# Exploration of the Energetic Stability Landscape of Free-Standing 2D Perovskites

In [1]:
try:
    from ase.db import connect
except ImportError:
    !pip install ase
    from ase.db import connect

try:
    import plotly 
except ImportError:
    !pip install plotly
    
from utils import *

In [63]:
#Getting the data from the database into a form that can be plotted 

#data that is characterised by the orientations and surface terminations of the 2D structures
two_d_100_AO_en_diff = []
two_d_100_BO2_en_diff  = []
two_d_110_ABO_en_diff = []
two_d_110_O2_en_diff  = []
two_d_111_AO3_en_diff = []
two_d_111_B_en_diff   = []

two_d_100_AO_keys = []
two_d_100_BO2_keys  = []
two_d_110_ABO_keys = []
two_d_110_O2_keys  = []
two_d_111_AO3_keys = []
two_d_111_B_keys   = []

bulk_100_AO_en_diff = []
bulk_100_BO2_en_diff  = []
bulk_110_ABO_en_diff = []
bulk_110_O2_en_diff  = []
bulk_111_AO3_en_diff = []
bulk_111_B_en_diff   = []

#data that is characterised by the chemical categories of the perovskite compounds 
#and the nature of the surface terminations
class_I_A_rich_term_en_diff = []
class_I_B_rich_term_en_diff = []
class_II_A_rich_term_en_diff = []
class_II_B_rich_term_en_diff = []
class_III_A_rich_term_en_diff = []
class_III_B_rich_term_en_diff = []
class_IV_A_rich_term_en_diff = []
class_IV_B_rich_term_en_diff = []

class_I_A_rich_term_bulk_en_diff = []
class_I_B_rich_term_bulk_en_diff = []
class_II_A_rich_term_bulk_en_diff = []
class_II_B_rich_term_bulk_en_diff = []
class_III_A_rich_term_bulk_en_diff = []
class_III_B_rich_term_bulk_en_diff = []
class_IV_A_rich_term_bulk_en_diff = []
class_IV_B_rich_term_bulk_en_diff = []

class_I_A_rich_term_keys = []
class_I_B_rich_term_keys = []
class_II_A_rich_term_keys = []
class_II_B_rich_term_keys = []
class_III_A_rich_term_keys = []
class_III_B_rich_term_keys = []
class_IV_A_rich_term_keys = []
class_IV_B_rich_term_keys = []



bulk_db=connect('2dpv_set_bulk.db')

for orientation in ['100','110','111']: #different orientations of the 2D perovskites 
    for term_type_id, term_type in enumerate(termination_types[orientation]): #two different terminations for a given 2D perovskites 
        
        #Connect to the database holding the data for this type of 2D perovskites. We don't have a grand 
        #database for all the calculation data because the problem of github holding a single large file.
        this_db = connect('2dpv_set_'+orientation+'_'+term_type+'.db')
        
        for i in range(len(A_site_list)): #loop through the chemical space for the perovskites investigated here
            for count_a, a in enumerate(A_site_list[i]):
                    for b in B_site_list[i]:
                        for c in C_site_list[i]:
                            system_name=a+b+c
                            
                            #getting the formation energy for the bulk cubic perovskite structures in
                            #the space group of Pm\bar{3}m
                            uid = system_name + '3_pm3m'
                            row = bulk_db.get(selection=[('uid', '=', uid)])
                            pm3m_formation_e = row.key_value_pairs['formation_energy']
                            
                            lowest_relaxed = None
                            
                            #getting the lowest energy amongst all the relaxed random structures
                            lowest_relaxed = -100000
                            for k in range(10):
                                uid = system_name + '3_random_str_' + str(k + 1)
                                try:
                                    row = bulk_db.get(selection=[('uid', '=', uid)])
                                    randomised_formation_e = row.key_value_pairs['formation_energy']
                                    if (randomised_formation_e > lowest_relaxed):  
                                        lowest_relaxed = randomised_formation_e
                                except KeyError:
                                    continue
                            
                            twod_formation_e = None
                            #getting the formation energy for the corresponding 2D free-standing perovskites
                            uid = system_name + '3_' + str(orientation) + "_" + str(term_type) + "_3"
                            try:
                                row = this_db.get(selection=[('uid', '=', uid)])
                                twod_formation_e = row.key_value_pairs['formation_energy']
                            except KeyError:
                                print('WARNING: No value for ' + uid + ' Skip!')
                                continue
                            
                            if (twod_formation_e is None) or (lowest_relaxed is None):
                                continue
                                
                            de_2d_bulk = twod_formation_e - pm3m_formation_e
                            de_bulk_convex = pm3m_formation_e - lowest_relaxed
                            
                            if (orientation,term_type) == ('100','AO'):
                                two_d_100_AO_en_diff.append(de_2d_bulk)
                                bulk_100_AO_en_diff.append(de_bulk_convex)
                                two_d_100_AO_keys.append(uid)
                            elif (orientation,term_type) == ('100','BO2'):
                                two_d_100_BO2_en_diff.append(de_2d_bulk)
                                bulk_100_BO2_en_diff.append(de_bulk_convex)
                                two_d_100_BO2_keys.append(uid)
                            elif (orientation,term_type) == ('110','ABO'):
                                two_d_110_ABO_en_diff.append(de_2d_bulk)
                                bulk_110_ABO_en_diff.append(de_bulk_convex)
                                two_d_110_ABO_keys.append(uid)
                            elif (orientation,term_type) == ('110','O2'):
                                two_d_110_O2_en_diff.append(de_2d_bulk)
                                bulk_110_O2_en_diff.append(de_bulk_convex)
                                two_d_110_O2_keys.append(uid)
                            elif (orientation,term_type) == ('111','AO3'):
                                two_d_111_AO3_en_diff.append(de_2d_bulk)
                                bulk_111_AO3_en_diff.append(de_bulk_convex)
                                two_d_111_AO3_keys.append(uid)
                            elif (orientation,term_type) == ('111','B'):
                                two_d_111_B_en_diff.append(de_2d_bulk)
                                bulk_111_B_en_diff.append(de_bulk_convex)
                                two_d_111_B_keys.append(uid)
                                
                            if (i==0) and (term_type in ['AO','O2','AO3']):
                                class_I_A_rich_term_en_diff.append(de_2d_bulk)
                                class_I_A_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_I_A_rich_term_keys.append(uid)
                            elif (i==0) and (term_type in ['BO2','ABO','B']):
                                class_I_B_rich_term_en_diff.append(de_2d_bulk)
                                class_I_B_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_I_B_rich_term_keys.append(uid)
                            elif (i==1) and (term_type in ['AO','O2','AO3']):
                                class_II_A_rich_term_en_diff.append(de_2d_bulk)
                                class_II_A_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_II_A_rich_term_keys.append(uid)
                            elif (i==1) and (term_type in ['BO2','ABO','B']):
                                class_II_B_rich_term_en_diff.append(de_2d_bulk)
                                class_II_B_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_II_B_rich_term_keys.append(uid)
                            elif (i==2) and (term_type in ['AO','O2','AO3']):
                                class_III_A_rich_term_en_diff.append(de_2d_bulk)
                                class_III_A_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_III_A_rich_term_keys.append(uid)
                            elif (i==2) and (term_type in ['BO2','ABO','B']):
                                class_III_B_rich_term_en_diff.append(de_2d_bulk)
                                class_III_B_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_III_B_rich_term_keys.append(uid)
                            elif (i==3) and (term_type in ['AO','O2','AO3']):
                                class_IV_A_rich_term_en_diff.append(de_2d_bulk)
                                class_IV_A_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_IV_A_rich_term_keys.append(uid)
                            elif (i==3) and (term_type in ['BO2','ABO','B']):
                                class_IV_B_rich_term_en_diff.append(de_2d_bulk)
                                class_IV_B_rich_term_bulk_en_diff.append(de_bulk_convex)
                                class_IV_B_rich_term_keys.append(uid)

In this interactive plot, we provide an interactive navigation of the formation energy landscape of the ultrathin free-standing 2D perovskites, similar to Fig. 3 in the manuscript. Here, the energy differences between the free-standing 2D perovskites with 3 atomic-layer thickness $(n=3)$ and bulk cubic perovskites $(E_f^{2D,n=3}-E_f^{Pm\bar{3}m})$ are compared against the minimum value of the 'convex hull energy' $(\min[E_f^{Pm\bar{3}m}-E_f^{full-relax}])$ for the bulk cubic perovskites. 2D structures with A-/B-rich terminating surfaces are marked with filled/empty symbols, respectively.

In the following plot, we provide a handle to explore the landscape of 2D perovskites grouped by different combinations of orientations and surface terminations.

In [60]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=two_d_100_AO_en_diff,
                         y=bulk_100_AO_en_diff,
                         name="[100]//AX",
                         text=two_d_100_AO_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#07000E',line=dict(color='#07000E',width=1))))

fig.add_trace(go.Scatter(x=two_d_100_BO2_en_diff,
                         y=bulk_100_BO2_en_diff,
                         name="[100]//BX2",
                         text=two_d_100_BO2_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#07000E')))

fig.add_trace(go.Scatter(x=two_d_110_ABO_en_diff,
                         y=bulk_110_ABO_en_diff,
                         name="[110]//ABX",
                         text=two_d_110_ABO_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#D75404',line=dict(color='#D75404',width=1))))

fig.add_trace(go.Scatter(x=two_d_110_O2_en_diff,
                         y=bulk_110_O2_en_diff,
                         name="[110]//X2",
                         text=two_d_110_O2_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#D75404')))

fig.add_trace(go.Scatter(x=two_d_111_AO3_en_diff,
                         y=bulk_111_AO3_en_diff,
                         name="[111]//AX3",
                         text=two_d_111_AO3_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#D50B53',line=dict(color='#D50B53',width=1))))

fig.add_trace(go.Scatter(x=two_d_111_B_en_diff,
                         y=bulk_111_B_en_diff,
                         name="[111]//B",
                         text=two_d_111_B_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#D50B53')))



fig.update_xaxes(title_text="$E_f^{2D,n=3}-E_f^{Pm\\bar{3}m}$")
fig.update_yaxes(title_text='$\\min[E_f^{Pm\\bar{3}m}-E_f^{full-relax}]$')

fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.update_layout(yaxis_range=[-0.7,0.5])
fig.update_layout(xaxis_range=[-1.1,2.1])
fig.update_layout(width=900, height=800,
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True]},
                          {'showlegend':True}]),
            dict(label = '[100]//AX',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False]},
                          {'showlegend':True}]),
            dict(label = '[100]//BX2',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False]},
                          {'showlegend':True}]),
            dict(label = '[110]//ABX',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False]},
                          {'showlegend':True}]),
            dict(label = '[110]//X2',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False]},
                          {'showlegend':True}]),
             dict(label = '[111]//AX3',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False]},
                          {'showlegend':True}]),
            dict(label = '[111]//B',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True]},
                          {'showlegend':True}])])
    )])

In the following plot, we provide a handle to explore the landscape of 2D perovskites grouped by different chemistry of the perovskites.

In [81]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=class_I_A_rich_term_en_diff,
                         y=class_I_A_rich_term_bulk_en_diff,
                         name="$A^{I}B^{II}_{M}X_{3}-(AX,X_{2},AX_{3})$",
                         text=class_I_A_rich_term_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#07000E',line=dict(color='#07000E',width=1))))
fig.add_trace(go.Scatter(x=class_I_B_rich_term_en_diff,
                         y=class_I_B_rich_term_bulk_en_diff,
                         name="$A^{I}B^{II}_{M}X_{3}-(BX_{2},ABX,B)$",
                         text=class_I_B_rich_term_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#07000E')))

fig.add_trace(go.Scatter(x=class_II_A_rich_term_en_diff,
                         y=class_II_A_rich_term_bulk_en_diff,
                         name="$A^{I}B^{II}_{TM}X_{3}-(AX,X_{2},AX_{3})$",
                         text=class_II_A_rich_term_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#D75404',line=dict(color='#D75404',width=1))))
fig.add_trace(go.Scatter(x=class_II_B_rich_term_en_diff,
                         y=class_II_B_rich_term_bulk_en_diff,
                         name="$A^{I}B^{II}_{TM}X_{3}-(BX_{2},ABX,B)$",
                         text=class_II_B_rich_term_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#D75404')))

fig.add_trace(go.Scatter(x=class_III_A_rich_term_en_diff,
                         y=class_III_A_rich_term_bulk_en_diff,
                         name="$A^{I}B^{IV}C_{3}-(AC,C_{2},AC_{3})$",
                         text=class_III_A_rich_term_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#522E75',line=dict(color='#522E75',width=1))))
fig.add_trace(go.Scatter(x=class_III_B_rich_term_en_diff,
                         y=class_III_B_rich_term_bulk_en_diff,
                         name="$A^{I}B^{IV}C_{3}-(BX_{2},ABX,B)$",
                         text=class_III_B_rich_term_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#522E75')))

fig.add_trace(go.Scatter(x=class_IV_A_rich_term_en_diff,
                         y=class_IV_A_rich_term_bulk_en_diff,
                         name="$A^{I}B^{X}C_{3}-(AC,C_{2},AC_{3})$",
                         text=class_IV_A_rich_term_keys,
                         opacity=0.7,
                         marker=dict(size=8,color='#D50B53',line=dict(color='#D50B53',width=1))))
fig.add_trace(go.Scatter(x=class_IV_B_rich_term_en_diff,
                         y=class_IV_B_rich_term_bulk_en_diff,
                         name="$A^{I}B^{X}C_{3}-(BX_{2},ABX,B)$",
                         text=class_IV_B_rich_term_keys,
                         opacity=0.7,
                         marker=dict(symbol='circle-open',size=8,color='#D50B53')))

fig.update_xaxes(title_text="$E_f^{2D,n=3}-E_f^{Pm\\bar{3}m}$")
fig.update_yaxes(title_text='$\\min[E_f^{Pm\\bar{3}m}-E_f^{full-relax}]$')

fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.update_layout(yaxis_range=[-0.7,0.5])
fig.update_layout(xaxis_range=[-1.1,2.1])
fig.update_layout(width=1000, height=800,
        updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True]},
                          {'showlegend':True}]),
            dict(label = "Class I/A-rich-term",
                  method = 'update',
                  args = [{'visible': [True, False, False, False, False, False, False, False]},
                          {'showlegend':True}]),
            dict(label = "Class I/B-rich-term",
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, False, False, False]},
                          {'showlegend':True}]),
            dict(label = "Class II/A-rich-term",
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, False, False]},
                          {'showlegend':True}]),
            dict(label = "Class II/B-rich-term",
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, False]},
                          {'showlegend':True}]),
            dict(label = "Class III/A-rich-term",
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False, False, False]},
                          {'showlegend':True}]),
            dict(label = "Class III/B-rich-term",
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True, False, False]},
                          {'showlegend':True}]),
            dict(label = "Class IV/A-rich-term",
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, True, False]},
                          {'showlegend':True}]),
            dict(label = "Class IV/B-rich-term",
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, True]},
                          {'showlegend':True}])
            ]))])